In [104]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [105]:
import sys
import os
import httpx

In [106]:
def make_api_request(api_endpoint, params):

    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None

In [107]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY="17c09553207c6d6e7bab10003e604aa8"
BINANCE_API_KEY="17754325821w1b1e2khd34561hc54hh1"

def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

In [108]:
data_downloaded = await get_historical_price_full_stock("AAPL")
data = data_downloaded['historical']
data = pd.DataFrame(data)
data.columns = [x.title() for x in data.columns] #uppercase first letter
data.drop(['Adjclose','Unadjustedvolume', 'Change', 'Changepercent', 'Vwap', 'Label','Changeovertime'], axis=1,inplace=True)
data['Date']=pd.to_datetime(data['Date'])
data.set_index('Date',inplace=True) #date needs to be set as index!
data=data.iloc[::-1] #to reverse the order of the dataframe
data

TypeError: object dict can't be used in 'await' expression

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as taPanda
class Ema(Strategy):
    n1=20
    n2=80
    n3=150
    def init(self):
        close = self.data.Close
        self.ema20 = self.I(taPanda.ema, close.s, self.n1)
        self.ema80 = self.I(taPanda.ema, close.s, self.n2)
        self.ema150 = self.I(taPanda.ema, close.s, self.n3)

    def next(self):
        price = self.data.Close
        if crossover(self.ema20, self.ema80):
            self.position.close()
            self.buy(sl=0.90*price,tp=1.25*price)

        elif crossover(self.ema80, self.ema20):
            self.position.close()
            self.sell(sl=1.10*price,tp=0.75*price)


bt = Backtest(data, Ema,cash=100000, commission=.002, exclusive_orders=True)
output = bt.run()

In [ ]:
data = pd.DataFrame(pd.DataFrame(output).T)
data['Sizes'] = output._trades['Size'].to_string().replace('/n',',')
data['EntryTimes'] = output._trades['EntryTime'].to_string().replace('/n',',')
data['ExitTimes'] =  output._trades['ExitTime'].to_string().replace('/n',',')
data['Durations'] = pd.DataFrame([str(output._trades['ExitTime'][i] - output._trades['EntryTime'][i]) for i in range(len(output._trades['ExitTime']))]).to_string().replace('/n',',')
data['EntryPrices'] = output._trades['EntryPrice'].to_string().replace('/n',',')
data['ExitPrices'] = output._trades['ExitPrice'].to_string().replace('/n',',')
data['EntryBars'] = output._trades['EntryBar'].to_string().replace('/n',',')
data['ExitBars'] = output._trades['ExitBar'].to_string().replace('/n',',')
data['PnLs'] = output._trades['PnL'].to_string().replace('/n',',')
data['ReturnPcTs'] = output._trades['ReturnPct'].to_string().replace('/n',',')

In [136]:
def run_backtests(instruments: list, strategy: Strategy):
    """Run backtests for a list of instruments using a specified strategy."""

    outputs = []
    optimise = []

    # Loop through the instruments
    for instrument in instruments:
        # Run API calls or load CSVs to get data
        data = get_historical_price_full_stock(instrument)
        data = data['historical']
        data = pd.DataFrame(data)
        data.columns = [x.title() for x in data.columns] #uppercase first letter
        data.drop(['Adjclose','Unadjustedvolume', 'Change', 'Changepercent', 'Vwap', 'Label','Changeovertime'], axis=1,inplace=True)
        data['Date']=pd.to_datetime(data['Date'])
        data.set_index('Date',inplace=True) #date needs to be set as index!
        data=data.iloc[::-1] #to reverse the order of the dataframe
        data

        # Create a backtest for the instrument using the specified strategy
        bt = Backtest(data, Ema,cash=100000, commission=.002, exclusive_orders=True)

        output = bt.run()

        output = pd.DataFrame(pd.DataFrame(output).T)

        # put instrument name in the output
        output['Instrument'] = instrument
        output['Strategy'] = strategy.__name__
        output.pop('_strategy')
        output['StrategyParameters'] = strategy.__dict__

        # output['Sizes'] = output._trades['Size'].to_string().replace('/n',',')
        # output['EntryTimes'] = output._trades['EntryTime'].to_string().replace('/n',',')
        # output['ExitTimes'] =  output._trades['ExitTime'].to_string().replace('/n',',')
        # output['Durations'] = pd.DataFrame([str(output._trades['ExitTime'][i] - output._trades['EntryTime'][i]) for i in range(len(output._trades['ExitTime']))]).to_string().replace('/n',',')
        # output['EntryPrices'] = output._trades['EntryPrice'].to_string().replace('/n',',')
        # output['ExitPrices'] = output._trades['ExitPrice'].to_string().replace('/n',',')
        # output['EntryBars'] = output._trades['EntryBar'].to_string().replace('/n',',')
        # output['ExitBars'] = output._trades['ExitBar'].to_string().replace('/n',',')
        # output['PnLs'] = output._trades['PnL'].to_string().replace('/n',',')
        # output['ReturnPcTs'] = output._trades['ReturnPct'].to_string().replace('/n',',')

        outputs.append(output)

        start = output['Start'].to_string().strip().split()[1]
        end = output['End'].to_string().strip().split()[1]

        fileNameOutput=f"output/raw/plot/{bt._strategy.__name__}-{instrument}-{start}-{end}.html"
        bt.plot(filename=fileNameOutput, open_browser=False)
        output.to_csv(f"output/raw/{bt._strategy.__name__}-{instrument}-{start}-{end}.csv")


        # Define the parameter ranges to optimize over
        param_ranges = {
            'n1': range(20, 80, 10),
            'n2': range(80, 160, 10),
            'n3': range(150, 300, 10)
        }

        # Define the metric to optimize for
        metric = 'Equity Final [$]'

        # Run the optimization
        results = bt.optimize(
            n1=param_ranges['n1'],
            n2=param_ranges['n2'],
            n3=param_ranges['n3'],
            maximize=metric,
            max_tries=100,
            random_state=42
        )

        results = pd.DataFrame(pd.DataFrame(results).T)


        # Save the results to a CSV file
        results.to_csv(f'output/raw/optimise/{strategy.__name__}_optimise.csv')

        # Append the results to the list of outputs
        optimise.append(results)


    # combine all the dataframes into one
    datas_frame = pd.concat(outputs)
    optimise_frame = pd.concat(optimise)

    # Save the data to a CSV file
    datas_frame.to_csv(f'output/{strategy.__name__}.csv')
    optimise_frame.to_csv(f'output/{strategy.__name__}_optimise.csv')


    # Return the list of datas
    return outputs


In [137]:
run_backtests(["AAPL", "MSFT", "AMZN", "GOOG", "FB", "TSLA", "NFLX", "NVDA", "PYPL", "ADBE"], Ema)

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


...

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

[       Start        End  Duration Exposure Time [%] Equity Final [$]  \
 0 2018-03-29 2023-03-28 1825 days         53.974563     132498.83935   
 
   Equity Peak [$] Return [%] Buy & Hold Return [%] Return (Ann.) [%]  \
 0    175332.16607  32.498839            275.849326           5.79893   
 
   Volatility (Ann.) [%]  ... Max. Trade Duration Avg. Trade Duration  \
 0             23.375625  ...            158 days             61 days   
 
   Profit Factor Expectancy [%]       SQN  \
 0      1.648512       2.634412  0.464416   
 
                                        _equity_curve  \
 0                    Equity  DrawdownPct Drawdown...   
 
                                              _trades Instrument Strategy  \
 0      Size  EntryBar  ExitBar  EntryPrice  ExitP...       AAPL      Ema   
 
   StrategyParameters  
 0                NaN  
 
 [1 rows x 32 columns],
        Start        End  Duration Exposure Time [%] Equity Final [$]  \
 0 2018-03-29 2023-03-28 1825 days         40

In [138]:
run_backtests(get_financial_statements_lists(), Ema)

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


...

/tmp/ipykernel_133871/3082792243.py:21: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Ema,cash=100000, commission=.002, exclusive_orders=True)
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/antoine/.local/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
